Import Libraries

In [3]:
import psycopg2
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

Connect to PostgreSQL and create tables

In [4]:
try:
    conn = psycopg2.connect(
        dbname="api_development_db",
        user="postgres",
        password="semah",
        host="localhost",
        port="5432"
    )
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS job_descriptions (
                      id SERIAL PRIMARY KEY,
                      filename TEXT NOT NULL,
                      text TEXT NOT NULL)''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS cvs (
                      id SERIAL PRIMARY KEY,
                      filename TEXT NOT NULL,
                      text TEXT NOT NULL,
                      qualifications TEXT NOT NULL,
                      skills TEXT NOT NULL,
                      experience TEXT NOT NULL)''')
    conn.commit()
    logger.info("Connected to PostgreSQL and tables created")
except Exception as e:
    logger.error(f"Error connecting to PostgreSQL: {e}")
    raise

INFO:__main__:Connected to PostgreSQL and tables created


Test storing a sample job description

In [5]:
def store_job_description(filename, text):
    cursor.execute("INSERT INTO job_descriptions (filename, text) VALUES (%s, %s)", (filename, text))
    conn.commit()
    cursor.execute("SELECT LASTVAL()")
    job_id = cursor.fetchone()[0]
    logger.info(f"Stored job with ID: {job_id}")
    return job_id

job_id = store_job_description("sample_job.pdf", "Sample job description for a juriste role")

INFO:__main__:Stored job with ID: 11


Test storing a sample CV

In [6]:
def store_cv(filename, text, qualifications, skills, experience):
    cursor.execute("INSERT INTO cvs (filename, text, qualifications, skills, experience) VALUES (%s, %s, %s, %s, %s)", 
                   (filename, text, ",".join(qualifications), ",".join(skills), ",".join(experience)))
    conn.commit()
    cursor.execute("SELECT LASTVAL()")
    cv_id = cursor.fetchone()[0]
    logger.info(f"Stored CV with ID: {cv_id}")
    return cv_id

cv_id = store_cv("juriste_cv.png", "CV for a juriste", ["law degree"], ["negotiation"], ["5 years"])

INFO:__main__:Stored CV with ID: 2


Retrieve and display all stored jobs

In [7]:
cursor.execute("SELECT * FROM job_descriptions")
jobs = cursor.fetchall()
logger.info(f"Found {len(jobs)} jobs")
for job in jobs:
    print(f"Job ID: {job[0]}, Filename: {job[1]}, Text: {job[2]}")

INFO:__main__:Found 11 jobs


Job ID: 1, Filename: job4.docx, Text: Job 4: Legal Assistant position offering 5 years of experience in case preparation and analysis.
Job ID: 2, Filename: job4.pdf, Text: Job 4: Legal Assistant position offering 5 years of experience in case preparation and analysis.
Job ID: 3, Filename: job5.docx, Text: Job 5: Attorney with 1 years of practice in intellectual property law and client counseling.
Job ID: 4, Filename: job5.pdf, Text: Job 5: Attorney with 1 years of practice in intellectual property law and client counseling.
Job ID: 5, Filename: job6.docx, Text: Job 6: Legal Analyst position requiring 2 years of experience in contract law and negotiation skills.
Job ID: 6, Filename: job7.pdf, Text: Job 7: Paralegal role with expertise in legal research and 3 years of drafting contracts.
Job ID: 7, Filename: job8.docx, Text: Job 8: Corporate Lawyer needed with 4 years in compliance and litigation management.
Job ID: 8, Filename: job8.pdf, Text: Job 8: Corporate Lawyer needed with 4 years

Retrieve and display all stored CVs

In [8]:
cursor.execute("SELECT * FROM cvs")
cvs = cursor.fetchall()
logger.info(f"Found {len(cvs)} CVs")
for cv in cvs:
    print(f"CV ID: {cv[0]}, Filename: {cv[1]}, Qualifications: {cv[3].split(',')}")

INFO:__main__:Found 2 CVs


CV ID: 1, Filename: juriste.png, Qualifications: ['jurisprudence', 'degree', 'law']
CV ID: 2, Filename: juriste_cv.png, Qualifications: ['law degree']


Close the database connection

cursor.close()
conn.close()
logger.info("Database connection closed")